# Semantic Segmentation of Persian Garden Images
## Case Study: Fin Garden, Kashan

This notebook implements a semantic segmentation pipeline using a U-Net
architecture (PyTorch) for Persian garden images.

The dataset consists of RGB images and RGB-encoded segmentation masks
annotated in Roboflow.

Classes:
- Trees
- Buildings
- Sky
- Steps
- Cover Plants
- Water


In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
# Paths & file pairing
IMAGE_DIR = "/Users/hessam/Documents/GitHub/Persian_Garden_Segmentation_Analysis/Data/image"
MASK_DIR = "/Users/hessam/Documents/GitHub/Persian_Garden_Segmentation_Analysis/Data/mask"

image_files = sorted(os.listdir(IMAGE_DIR))
mask_files = sorted(os.listdir(MASK_DIR))

print(f"Images: {len(image_files)}")
print(f"Masks : {len(mask_files)}")

assert len(image_files) == len(mask_files), "Image–mask count mismatch"


In [ ]:
# Visual sanity check
idx = 0

img = Image.open(os.path.join(IMAGE_DIR, image_files[idx])).convert("RGB")
mask = Image.open(os.path.join(MASK_DIR, mask_files[idx])).convert("RGB")

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title("Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(mask)
plt.title("RGB Mask")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(img)
plt.imshow(mask, alpha=0.5)
plt.title("Overlay")
plt.axis("off")

plt.show()


In [1]:
# Unique RGB colors
mask_np = np.array(mask)
unique_colors = np.unique(mask_np.reshape(-1, 3), axis=0)

print("Unique RGB colors in this mask:")
unique_colors


NameError: name 'np' is not defined